# Structbio2022 - Practical day 03

## Analysis of MD simulations - Part 2

Document written by [Adrián Diaz](mailto:adrian.diaz@vub.be) & [David Bickel](mailto:david.bickel@vub.be).

<br/>

Now, after we did some initial analyses and verified that nothing strange happend, let's address our actual research question.

> ***Have a quick look at the crystal structures of the ccdB:ccdB in free and ccdA-bound form.***
>
> Which of the structures is the ccdA bound form?

<br/>

In the last steps, we did some general analysis on our trajectory to verify that the simulations ran smoothely. Now let's proceed to target our actual research question.

***

Before we start, maybe let's remind ourselves of the question we want to tackle.

> ***Does ccdA change the way the ccdB:ccdB dimers interact?***
>
> Which of the structures is the ccdA bound form? Can you see the angle shift between the ccdB-monomers?

In [ ]:
import nglview as nv

ccdBB  = nv.FileStructure("dimer_analysis/1x75_superimposed.pdb")
ccdBBA = nv.FileStructure("dimer_analysis/3hpw_superimposed.pdb")

view = nv.show_file(ccdBB, default_representation=False)
view.add_component(ccdBBA, default_representation=False)

# ccdB:ccdB dimer
view[0].add_cartoon(selection=":B", color=0x337fcc)
view[0].add_cartoon(selection=":A", color=0x7ba9d8)
# ccdB:ccdB.ccdA complex
view[1].add_cartoon(selection=":B", color=0x7fff00)
view[1].add_cartoon(selection=":A or :X", color=0xd2ffa6)
view.center()
view

If both structures are aligned on one of the monomers, the other one seems lightly offset.

Let's see if we can quantify that. How would you do that?

In [ ]:
import MDAnalysis as mda
from dimer_analysis import DimerAnalysis

# Analyse angle for ccdB:ccdB complex
ccdBB  = mda.Universe("dimer_analysis/1x75_superimposed.pdb")
analyBB = DimerAnalysis(ccdBB.select_atoms("chainID A"), ccdBB.select_atoms("chainID B"))
analyBB.run()

# Analyse angle for ccdB:ccdB:ccdA complex
ccdBBA = mda.Universe("dimer_analysis/3hpw_superimposed.pdb")
analyBBA = DimerAnalysis(ccdBBA.select_atoms("chainID A"), ccdBBA.select_atoms("chainID B"))
analyBBA.run()

print("Angle between dimers (ccdB:ccdB     ): {0:8.3f}°".format(*analyBB.results.rotation_angle))
print("Angle between dimers (ccdB:ccdB:ccdA): {0:8.3f}°".format(*analyBBA.results.rotation_angle))

*** 

Now, let's run the same analysis on every frame of our trajectory:

In [ ]:
import MDAnalysis as mda
import pandas as pd
from dimer_analysis import DimerAnalysis, GmxNDX

# Load the trajectory
md_simulation = mda.Universe("dry_md.tpr", "dry_md.xtc")

# load the index file
ndx = GmxNDX("index.ndx")

# Select first ccdB monomer
selection_1 = ndx.select_atomgroup(md_simulation)

# Select second ccdB monomer
selection_2 = ndx.select_atomgroup(md_simulation)

# Run the analysis
analy = DimerAnalysis(selection_1, selection_2)
analy.run()

# Process results
data = pd.DataFrame({"Time": analy.results.time, "DimerRotation": analy.results.rotation_angle})
data.head()

In [ ]:
data.describe()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde
%matplotlib inline

# Plot data
fig, (ax0, ax1) = plt.subplots(1,2, figsize=(4,2), dpi=150, sharey=True,
                        gridspec_kw={"width_ratios": [3,1], "wspace": .05})
# Time series
ax0.grid(True, "both", "both", linewidth=.5, alpha=.5)
ax0.plot(data["Time"], data["DimerRotation"], linewidth=.7)
ax0.set_ylabel("Rotation | dg")
ax0.set_xlabel("Time | ns")

# Density distribution
kde   = gaussian_kde(data["DimerRotation"])
rvals = np.linspace(np.min(data["DimerRotation"]), np.max(data["DimerRotation"]), 100)
ax1.grid(True, "both", "both", linewidth=.5, alpha=.5)
ax1.fill_betweenx(rvals, np.zeros(rvals.shape), kde(rvals))

plt.show()